# Tutorial - Spark in Python
This notebook is designed to introduce some basic concepts and help get you familiar with using Spark in Python.   

In this notebook, we will load and explore the mtcars dataset. Specifically, this tutorial covers:

1. Loading data in memory
1. Creating SQLContext
1. Creating Spark DataFrame
1. Group data by columns 
1. Operating on columns
1. Running SQL Queries from a Spark DataFrame


## Loading in a DataFrame
To create a Spark DataFrame we load an external DataFrame, called `mtcars`. This DataFrame includes 32 observations on 11 variables.

[, 1]	mpg	Miles/(US) --> gallon  
[, 2]	cyl	--> Number of cylinders  
[, 3]	disp	--> Displacement (cu.in.)  
[, 4]	hp -->	Gross horsepower  
[, 5]	drat -->	Rear axle ratio  
[, 6]	wt -->	Weight (lb/1000)  
[, 7]	qsec -->	1/4 mile time  
[, 8]	vs -->	V/S  
[, 9]	am -->	Transmission (0 = automatic, 1 = manual)  
[,10]	gear -->	Number of forward gears  
[,11]	carb -->	Number of carburetors  

In [1]:
import pandas as pd
mtcars = pd.read_csv('https://ibm.box.com/shared/static/f1dhhjnzjwxmy2c1ys2whvrgz05d1pui.csv')

In [2]:
mtcars.head()

,car,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,Mazda RX4,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,Mazda RX4 Wag,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,Datsun 710,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,Hornet 4 Drive,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,Hornet Sportabout,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2


## Initialize SQLContext
To work with dataframes we need a SQLContext which is created using `SQLContext(sc)`. SQLContext uses SparkContext which has been already created, named `sc`. 

In [3]:
sqlContext = SQLContext(sc)

## Creating Spark DataFrames
With SQLContext and a loaded local DataFrame, we create a Spark DataFrame:

In [4]:
sdf = sqlContext.createDataFrame(mtcars) 
sdf.printSchema()

root
 |-- car: string (nullable = true)
 |-- mpg: double (nullable = true)
 |-- cyl: long (nullable = true)
 |-- disp: double (nullable = true)
 |-- hp: long (nullable = true)
 |-- drat: double (nullable = true)
 |-- wt: double (nullable = true)
 |-- qsec: double (nullable = true)
 |-- vs: long (nullable = true)
 |-- am: long (nullable = true)
 |-- gear: long (nullable = true)
 |-- carb: long (nullable = true)



## Displays the content of the DataFrame 


In [5]:
sdf.show(5)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 5 rows



## Selecting columns

In [6]:
sdf.select('mpg').show(5)

+----+
| mpg|
+----+
|21.0|
|21.0|
|22.8|
|21.4|
|18.7|
+----+
only showing top 5 rows



## Filtering Data
Filter the DataFrame to only retain rows with `mpg` less than 18

In [7]:
sdf.filter(sdf['mpg'] < 18).show(5)

+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
|        car| mpg|cyl| disp| hp|drat|  wt| qsec| vs| am|gear|carb|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
| Duster 360|14.3|  8|360.0|245|3.21|3.57|15.84|  0|  0|   3|   4|
|  Merc 280C|17.8|  6|167.6|123|3.92|3.44| 18.9|  1|  0|   4|   4|
| Merc 450SE|16.4|  8|275.8|180|3.07|4.07| 17.4|  0|  0|   3|   3|
| Merc 450SL|17.3|  8|275.8|180|3.07|3.73| 17.6|  0|  0|   3|   3|
|Merc 450SLC|15.2|  8|275.8|180|3.07|3.78| 18.0|  0|  0|   3|   3|
+-----------+----+---+-----+---+----+----+-----+---+---+----+----+
only showing top 5 rows



## Operating on Columns
SparkR also provides a number of functions that can directly applied to columns for data processing and aggregation. The example below shows the use of basic arithmetic functions to convert lb to metric ton.

In [8]:
sdf.withColumn('wtTon', sdf['wt'] * 0.45).show(6)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-------+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|  wtTon|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-------+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|  1.179|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|1.29375|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|  1.044|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|1.44675|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|  1.548|
|          Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|  1.557|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+-------+
only showing top 6 rows



In [9]:
sdf.show(6)

+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|              car| mpg|cyl| disp| hp|drat|   wt| qsec| vs| am|gear|carb|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
|        Mazda RX4|21.0|  6|160.0|110| 3.9| 2.62|16.46|  0|  1|   4|   4|
|    Mazda RX4 Wag|21.0|  6|160.0|110| 3.9|2.875|17.02|  0|  1|   4|   4|
|       Datsun 710|22.8|  4|108.0| 93|3.85| 2.32|18.61|  1|  1|   4|   1|
|   Hornet 4 Drive|21.4|  6|258.0|110|3.08|3.215|19.44|  1|  0|   3|   1|
|Hornet Sportabout|18.7|  8|360.0|175|3.15| 3.44|17.02|  0|  0|   3|   2|
|          Valiant|18.1|  6|225.0|105|2.76| 3.46|20.22|  1|  0|   3|   1|
+-----------------+----+---+-----+---+----+-----+-----+---+---+----+----+
only showing top 6 rows



## Grouping, Aggregation
Spark DataFrames support a number of commonly used functions to aggregate data after grouping. For example we can compute the average weight of cars by their cylinders as shown below:

In [10]:
sdf.groupby(['cyl'])\
.agg({"wt": "AVG"})\
.show(5)

+---+-----------------+
|cyl|          avg(wt)|
+---+-----------------+
|  4|2.285727272727273|
|  6|3.117142857142857|
|  8|3.999214285714286|
+---+-----------------+



In [11]:
# We can also sort the output from the aggregation to get the most common cars
car_counts = sdf.groupby(['cyl'])\
.agg({"wt": "count"})\
.sort("count(wt)", ascending=False)\
.show(5)


+---+---------+
|cyl|count(wt)|
+---+---------+
|  8|       14|
|  4|       11|
|  6|        7|
+---+---------+



### Running SQL Queries from Spark DataFrames
A Spark DataFrame can also be registered as a temporary table in Spark SQL and registering a DataFrame as a table allows you to run SQL queries over its data. The `sql` function enables applications to run SQL queries programmatically and returns the result as a DataFrame.



In [12]:
# Register this DataFrame as a table.
sdf.registerTempTable("cars")

# SQL statements can be run by using the sql method
highgearcars = sqlContext.sql("SELECT gear FROM cars WHERE cyl >= 4 AND cyl <= 9")
highgearcars.show(6)
    

+----+
|gear|
+----+
|   4|
|   4|
|   4|
|   3|
|   3|
|   3|
+----+
only showing top 6 rows



NOTE: This tutorial draws heavily from the original 
[Spark Quick Start Guide](http://spark.apache.org/docs/latest/quick-start.html)

## Want to learn more?

### Free courses on [Big Data University](https://bigdatauniversity.com/courses/what-is-spark/?utm_source=tutorial-spark-python&utm_medium=dswb&utm_campaign=bdu):

<a href="https://bigdatauniversity.com/courses/what-is-spark/?utm_source=tutorial-spark-python&utm_medium=dswb&utm_campaign=bdu"><img src = "https://ibm.box.com/shared/static/pxb2n9airrzrfola21zj5ssj7kndcp8m.png"> </a>

<h3>Authors:</h3>
<br>
<a href="https://ca.linkedin.com/in/saeedaghabozorgi">
    <div class="teacher-image" style="    float: left;
        width: 115px;
        height: 115px;
        margin-right: 10px;
        margin-bottom: 10px;
        border: 1px solid #CCC;
        padding: 3px;
        border-radius: 3px;
        text-align: center;"><img class="alignnone wp-image-2258 " src="https://ibm.box.com/shared/static/tyd41rlrnmfrrk78jx521eb73fljwvv0.jpg" alt="Saeed Aghabozorgi" width="178" height="178"/>
    </div>
</a>

<h4>Saeed Aghabozorgi</h4>
<p><a href="https://ca.linkedin.com/in/saeedaghabozorgi">Saeed Aghabozorgi</a>, PhD is a Data Scientist in IBM with a track record of developing enterprise level applications that substantially increases clients' ability to turn data into actionable knowledge. He is a researcher in data mining field and expert in developing advanced analytic methods like machine learning and statistical modelling on large datasets.</p>

<br>

<a href="https://ca.linkedin.com/in/polonglin">
    <div class="teacher-image" style="    float: left;
        width: 115px;
        height: 115px;
        margin-right: 10px;
        margin-bottom: 10px;
        border: 1px solid #CCC;
        padding: 3px;
        border-radius: 3px;
        text-align: center;"><img class="alignnone size-medium wp-image-2177" src="https://ibm.box.com/shared/static/2ygdi03ahcr97df2ofrr6cf8knq4kodd.jpg" alt="Polong Lin" width="300" height="300"/>
    </div>
</a>
<h4>Polong Lin</h4>
<p>
<a href="https://ca.linkedin.com/in/polonglin">Polong Lin</a> is a Data Scientist at IBM in Canada. Under the Emerging Technologies division, Polong is responsible for educating the next generation of data scientists through Big Data University. Polong is a regular speaker in conferences and meetups, and holds a M.Sc. in Cognitive Psychology.</p>

<hr>
Copyright &copy; 2016 [Big Data University](https://bigdatauniversity.com). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).​